# NYC Taxi Trip Duration Prediction

This notebook demonstrates a clean ML pipeline for predicting taxi trip durations using NYC Yellow Taxi data from October-November 2023.

## Pipeline Steps
1. Data preprocessing (using separate script)
2. Model training
3. Model evaluation
4. Results visualization

## 1. Setup and Imports

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings('ignore')

# Set plot style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

import sys
sys.path.append('..')

# Import our custom preprocessing functions

from taxi_ride.data.preprocess_data import (
    get_project_paths, 
    load_pickle, 
    dump_pickle,
    run_preprocessing
)

# Let's manually verify and set the correct paths
# Since we're in notebooks/, we need to go up one level to project root
NOTEBOOK_DIR = os.getcwd()
PROJECT_ROOT = os.path.dirname(NOTEBOOK_DIR)  # Go up one level

print(f"\nCurrent notebook directory: {NOTEBOOK_DIR}")
print(f"Project root: {PROJECT_ROOT}")

# Construct correct paths manually
CORRECT_RAW_DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'raw')
CORRECT_PROCESSED_DATA_DIR = os.path.join(PROJECT_ROOT, 'data', 'processed')

print(f"\nCorrected paths:")
print(f"Raw data directory: {CORRECT_RAW_DATA_DIR}")
print(f"Processed data directory: {CORRECT_PROCESSED_DATA_DIR}")

# Create directories if they don't exist
os.makedirs(CORRECT_RAW_DATA_DIR, exist_ok=True)
os.makedirs(CORRECT_PROCESSED_DATA_DIR, exist_ok=True)

print(f"\nDirectory structure:")
print(f"Raw data exists: {os.path.exists(CORRECT_RAW_DATA_DIR)}")
print(f"Processed data exists: {os.path.exists(CORRECT_PROCESSED_DATA_DIR)}")

## 2. Data Preprocessing

We'll use the `preprocess_data.py` script to load and prepare our data from URLs.

In [ ]:
run_preprocessing(
    raw_data_path=CORRECT_RAW_DATA_DIR,
    dest_path=CORRECT_PROCESSED_DATA_DIR,
    dataset="green",         # or "yellow"
    train_month="2023-10",
    val_month="2023-11",
    test_month="2023-12"
)



In [ ]:
# # 2. Download and Save Raw Data
# from taxi_ride.data.preprocess_data import (
#     get_data_path, 
#     load_parquet, 
#     save_parquet
# )

# # Configuration
# BASE_URL = "https://d37ci6vzurychx.cloudfront.net/trip-data"
# OUTPUT_DIR = CORRECT_RAW_DATA_DIR
# DATASET = "green"  # Change to "yellow" if needed
# MONTHS = ["2023-10", "2023-11", "2023-12"]

# #print(f"Output directory: {OUTPUT_DIR}")
# print(f"Dataset: {DATASET}")
# print(f"Months: {MONTHS}")

# # Ensure output directory exists
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# # Download each month's data using existing functions
# downloaded_files = []

# for month in MONTHS:
#     # Use get_data_path to construct the URL
#     url = get_data_path(BASE_URL, DATASET, month)
    
#     # Construct local output path
#     filename = f"{DATASET}_tripdata_{month}.parquet"
#     output_path = os.path.join(OUTPUT_DIR, filename)
    
#     print(f"\nProcessing {filename}...")
#     print(f"  URL: {url}")
#     #print(f"  Local path: {output_path}")
    
#     try:
#         # Use load_parquet to download data
#         df = load_parquet(url)
#         print(f"  ✓ Loaded data: {df.shape[0]:,} rows, {df.shape[1]} columns")
        
#         # Use save_parquet to save locally
#         save_parquet(df, output_path)
        
#         # Show basic info
#         print(f"  Columns: {len(df.columns)}")
        
#         downloaded_files.append(output_path)
        
#     except Exception as e:
#         print(f"  ✗ Error processing {filename}: {e}")

# print(f"\n✓ Download completed!")
# print(f"Files downloaded: {len(downloaded_files)}")

## 3. Load Preprocessed Data

In [ ]:
from taxi_ride.data.preprocess_data import load_pickle

PROCESSED_DATA_DIR = os.path.join("..", "data", "processed")

X_train, y_train = load_pickle(os.path.join(PROCESSED_DATA_DIR, "train.pkl"))
X_val, y_val = load_pickle(os.path.join(PROCESSED_DATA_DIR, "val.pkl"))
dv = load_pickle(os.path.join(PROCESSED_DATA_DIR, "dv.pkl"))

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Number of features: {len(dv.feature_names_)}")



## 4. Exploratory Data Analysis

In [ ]:
# Statistics on target variable
print("Training Data Duration Statistics:")
print(f"Mean: {y_train.mean():.2f} minutes")
print(f"Median: {np.median(y_train):.2f} minutes")
print(f"Std Dev: {y_train.std():.2f} minutes")
print(f"Min: {y_train.min():.2f} minutes")
print(f"Max: {y_train.max():.2f} minutes")

In [ ]:
# Distribution of trip durations
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(y_train, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Duration (minutes)')
plt.ylabel('Frequency')
plt.title('Distribution of Trip Durations')
plt.axvline(y_train.mean(), color='red', linestyle='--', label=f'Mean: {y_train.mean():.1f}')
plt.legend()

plt.subplot(1, 2, 2)
plt.boxplot(y_train, vert=True)
plt.ylabel('Duration (minutes)')
plt.title('Box Plot of Trip Durations')

plt.tight_layout()
plt.show()

## 5. Model Training

In [ ]:
# Train linear regression model
print("Training Linear Regression model...")
lr = LinearRegression()
lr.fit(X_train, y_train)
print("Training complete!")

## 6. Model Evaluation

In [ ]:
def evaluate_model(model, X, y, dataset_name=""):
    """Evaluate model and print metrics."""
    y_pred = model.predict(X)
    
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    mae = mean_absolute_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    print(f"\n{dataset_name} Metrics:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    
    return y_pred, rmse, mae, r2

# Evaluate on training set
y_train_pred, _, _, _ = evaluate_model(lr, X_train, y_train, "Training Set")

# Evaluate on validation set
y_val_pred, val_rmse, val_mae, val_r2 = evaluate_model(lr, X_val, y_val, "Validation Set")

## 7. Results Visualization

In [ ]:
# Compare predictions vs actual
plt.figure(figsize=(14, 6))

# Distribution comparison
plt.subplot(1, 2, 1)
sns.histplot(y_val, label='Actual', alpha=0.6, bins=50, color='blue')
sns.histplot(y_val_pred, label='Predicted', alpha=0.6, bins=50, color='red')
plt.xlabel('Duration (minutes)')
plt.ylabel('Frequency')
plt.title('Validation Set: Actual vs Predicted Distribution')
plt.legend()

# Scatter plot
plt.subplot(1, 2, 2)
# Sample for visualization (plot every 10th point)
sample_idx = np.arange(0, len(y_val), 10)
plt.scatter(y_val[sample_idx], y_val_pred[sample_idx], alpha=0.3, s=10)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2, label='Perfect prediction')
plt.xlabel('Actual Duration (minutes)')
plt.ylabel('Predicted Duration (minutes)')
plt.title(f'Validation Set: Predictions vs Actual\nR² = {val_r2:.4f}')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Residual analysis
residuals = y_val - y_val_pred

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Residual (minutes)')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.axvline(0, color='red', linestyle='--', label='Zero error')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(y_val_pred[sample_idx], residuals[sample_idx], alpha=0.3, s=10)
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Predicted Duration (minutes)')
plt.ylabel('Residual (minutes)')
plt.title('Residual Plot')

plt.tight_layout()
plt.show()

## 8. Model Summary

In [ ]:
print("="*50)
print("MODEL SUMMARY")
print("="*50)
print(f"\nModel Type: Linear Regression")
print(f"Number of Features: {X_train.shape[1]}")
print(f"Training Samples: {X_train.shape[0]:,}")
print(f"Validation Samples: {X_val.shape[0]:,}")
print(f"\nValidation Performance:")
print(f"  RMSE: {val_rmse:.4f} minutes")
print(f"  MAE:  {val_mae:.4f} minutes")
print(f"  R²:   {val_r2:.4f}")
print(f"\nMean Absolute Percentage Error: {(val_mae / y_val.mean() * 100):.2f}%")
print("="*50)

## 9. Save Model

In [ ]:
# Save the trained model
model_path = os.path.join(PROJECT_ROOT,"models", "linear_regression_model.pkl")
with open(model_path, "wb") as f_out:
    pickle.dump(lr, f_out)

print(f"Model saved to: {model_path}")